# YOLOモデルの再訓練

YOLOモデルを再訓練するためには、中程度および重度の事故ラベルが付けられた車の画像の準備されたデータセットが必要です。このデータセット（RoboFlowから取得したもの）には、すでに注釈付きの画像が含まれており、トレーニングセットとバリデーションセットに分割されています。これらのトレーニング/バリデーションセットを使用して、現在のYOLOモデルを再訓練します。トレーニングに必要なデータ構造に関するいくつかの情報は以下の通りです：

1. モデルに検出させたいオブジェクトのクラスは、0-`moderate`（中程度）および1-`severe`（重度）でエンコードされます。
2. データセットは自身のフォルダーにあり、その中に`train`と`valid`の2つのサブフォルダーがあります。それぞれのサブフォルダーの中には、`images`（画像）と`labels`（ラベル）の2つのサブフォルダーがあります。
3. 各画像には`labels`サブフォルダーに対応する注釈テキストファイルがあります。注釈テキストファイルは画像ファイルと同じ名前です。
4. データセットの記述子YAMLファイル（data.yaml）がデータセットを指し示し、それらの中のオブジェクトクラスを説明します。このYAMLファイルは、モデルの`train`メソッドに渡され、トレーニングプロセスが開始されます。

さあ、始めましょう！

In [ ]:
# このラボ用に設計されたWorkbenchイメージを使用していない場合は、以下の行のコメントを外して実行することで、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

次にYOLOモデル'yolo8m.pt'をロードします。

In [ ]:
# モデルをロード
model = YOLO('yolov8m.pt')  # プレトレーニングされたモデルをロード（トレーニングには推奨）

## トレーニングデータの取得

以下の2つのトレーニングデータセットを提供しています。ZIPファイルとして利用可能です：  
1) `accident-full.zip`   - モデルを完全に再訓練するために使用されます。
2) `accident-sample.zip` - モデルを部分的に再訓練するために使用され、完全に再訓練する時間がない場合に使用されます。

ワークショップでは、サンプルデータセットのみを使用します。

In [ ]:
# 特定のデータセットを取得するための関数
def retrieve_dataset(dataset_type):

    # ディレクトリが存在するか確認し、存在しない場合は作成
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # ファイルが存在するか確認し、存在しない場合はダウンロードして解凍
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("ファイルをダウンロード中...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("ファイルを解凍中...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("完了！")


dataset_type = 'sample'
# dataset_type = 'full' # フルデータセットを取得したい場合はこの行を代わりに使用
retrieve_dataset(dataset_type)

## YOLOモデルの再訓練

'エポック'が何であるかを理解することから始めましょう。機械学習モデルは、特定のデータセットをアルゴリズムに通して訓練されます。フルデータセットがアルゴリズムを通過するたびに、それが**エポック**を完了したと見なされます。各**エポック**はモデルの訓練をさらに洗練します。

以下のトレーニングコードでは、さまざまなパラメータを設定します：  
**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: プロセスをデモするために、1エポックのみを使用します。
- imgsz: これはモデルが与えられる画像のサイズです。
- batch: これはアルゴリズムが同時に処理する画像の数です。多いほど良い結果が得られますが、メモリを多く消費します。このワークショップではリソースが制約されており、これは単なるトレーニング例であるため、意図的に低い2に設定しています。

トレーニングの実行中、各**エポック**はトレーニングとバリデーションフェーズの両方の要約を表示します：行1と2はトレーニングフェーズの結果を示し、行3と4は各エポックのバリデーションフェーズの結果を示します。  

次のセルを実行してモデルの再訓練を開始しましょう！

In [ ]:
# モデルを訓練

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

**注意**：標準的な再訓練の後、結果に満足していれば、モデルをONNX形式にエクスポートできます。
(以下のコマンドで使用するトレーニングセッションをtrainXに置き換えてください)

`ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")`

## トレーニング結果の解釈

フルデータセットに対するトレーニングプロセスと結果の完全な説明は、ラボの指示にあります。

モデルを再訓練したので、車の事故画像に対してテストしてみましょう！

**ノートブック`04-04-accident-recog.ipynb`を開いてください**。